In [ ]:
import sys
sys.path.extend([ '../', '../../', '../../../'])
import torch
import dataloader as dl
from args import Args
# from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from model import HAN_ConEn, HAN_NetEn, EdgePredictor, HAN_classify
from train import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.dblp()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [ ]:
data = torch.load('../../data/dblp_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=4057,
    x=[4057, 334],
    y=[4057],
  },
  p={
    num_nodes=14328,
    x=[14328, 4231],
  },
  t={
    num_nodes=7723,
    x=[7723, 50],
  },
  c={ num_nodes=20 },
  a_embed={ x=[4057, 128] },
  p_embed={ x=[14328, 128] },
  t_embed={ x=[7723, 128] },
  c_embed={ x=[20, 128] },
  (a, walk, a)={ edge_index=[2, 10942] },
  (a, walk, p)={ edge_index=[2, 40537] },
  (a, walk, t)={ edge_index=[2, 32352] },
  (a, walk, c)={ edge_index=[2, 9147] },
  (p, walk, a)={ edge_index=[2, 68812] },
  (p, walk, p)={ edge_index=[2, 143279] },
  (p, walk, t)={ edge_index=[2, 114612] },
  (p, walk, c)={ edge_index=[2, 31949] },
  (t, walk, a)={ edge_index=[2, 31881] },
  (t, walk, p)={ edge_index=[2, 77052] },
  (t, walk, t)={ edge_index=[2, 61195] },
  (t, walk, c)={ edge_index=[2, 17021] },
  (c, walk, a)={ edge_index=[2, 144] },
  (c, walk, p)={ edge_index=[2, 200] },
  (c, walk, t)={ edge_index=[2, 160] },
  (c, walk, c)={ edge_index=[2, 17] },
  (a, to, p)={ edge_i

In [3]:
print(data['p','walk','a'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['a']['x'] = data['a']['x'].to(device)
data['p']['x'] = data['p']['x'].to(device)
data['t']['x'] = data['t']['x'].to(device)

data['c_embed']['x'] = data['c_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device) 
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 't']['edge_index'] = data['a', 'walk', 't']['edge_index'].to(device)
data['a', 'walk', 'c']['edge_index'] = data['a', 'walk', 'c']['edge_index'].to(device)

data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 't']['edge_index'] = data['p', 'walk', 't']['edge_index'].to(device)
data['p', 'walk', 'c']['edge_index'] = data['p', 'walk', 'c']['edge_index'].to(device)

data['t', 'walk', 'a']['edge_index'] = data['t', 'walk', 'a']['edge_index'].to(device)
data['t', 'walk', 'p']['edge_index'] = data['t', 'walk', 'p']['edge_index'].to(device)
data['t', 'walk', 't']['edge_index'] = data['t', 'walk', 't']['edge_index'].to(device)
data['t', 'walk', 'c']['edge_index'] = data['t', 'walk', 'c']['edge_index'].to(device)

data['c', 'walk', 'a']['edge_index'] = data['c', 'walk', 'a']['edge_index'].to(device)
data['c', 'walk', 'p']['edge_index'] = data['c', 'walk', 'p']['edge_index'].to(device)
data['c', 'walk', 't']['edge_index'] = data['c', 'walk', 't']['edge_index'].to(device)
data['c', 'walk', 'c']['edge_index'] = data['c', 'walk', 'c']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 't'].edge_index, data['a', 'walk', 'c'].edge_index ]

tensor([[    0,     0,     1,  ..., 14327, 14327, 14327],
        [  262,   888,   263,  ...,  2880,  3193,  1508]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, "\n", len(train_idx))
print("val_idx: ", val_idx, "\n", len(val_idx))
print("test_idx: ", test_idx, "\n", len(test_idx))
# print(c_num_mat)

[50, 20, 25, 30] 125
0 1197
1 745
2 1109
3 1006
train_idx:  [2160, 2082, 3345, 3102, 1805, 2090, 1833, 1846, 92, 181, 2706, 3676, 2299, 169, 139, 254, 1227, 140, 3693, 1236, 2335, 3989, 1800, 1688, 1439, 768, 3749, 3411, 2437, 3529, 2749, 277, 1122, 625, 1326, 3816, 332, 1934, 198, 2296, 1235, 2950, 880, 1944, 1857, 122, 241, 3776, 1294, 3095, 3106, 2526, 1133, 831, 2551, 1995, 706, 3498, 2214, 322, 792, 2200, 60, 2877, 1968, 3603, 1600, 2055, 3055, 276, 2287, 633, 2611, 390, 2083, 2125, 3439, 87, 1691, 3146, 2668, 1479, 2039, 3566, 3020, 2523, 2973, 2656, 2986, 3513, 91, 396, 1234, 1504, 1690, 2120, 3297, 1766, 3534, 347, 568, 2091, 2285, 2994, 684, 1954, 2998, 3716, 3256, 884, 648, 3444, 1811, 138, 3660, 2803, 2851, 3143, 3034, 4038, 1935, 3089, 1662, 3186, 3799] 
 125
val_idx:  [2663, 1493, 2965, 2061, 1692, 2937, 2036, 1054, 3234, 220, 2222, 3022, 2561, 3232, 3196, 2066, 229, 480, 3207, 1142, 33, 2096, 2527, 1296, 3652, 463, 2180, 1961, 1926, 1997, 1346, 4036, 1589, 203, 2255, 3912

In [ ]:
## HAN

encoder1 = HAN_ConEn(args.embed_dim, args, args.dropout)
encoder2 = HAN_NetEn(args.embed_dim, args.heads, args.dropout)
# encoder1 = torch.load('../pretrained_han/encoder1.pth')
# encoder2 = torch.load('../pretrained_han/encoder2.pth')
classifier = HAN_classify(args.embed_dim, args.heads, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_t = EdgePredictor(args.embed_dim)
decoder_c = EdgePredictor(args.embed_dim)
# decoder_a = torch.load('../pretrained_han/decoder_a.pth')
# decoder_p = torch.load('../pretrained_han/decoder_p.pth')
# decoder_t = torch.load('../pretrained_han/decoder_t.pth')
# decoder_c = torch.load('../pretrained_han/decoder_c.pth')

decoder_list = [decoder_a, decoder_p, decoder_t, decoder_c]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

In [ ]:
#HGNN 

# encoder1 = Het_ConEn(args.embed_dim, args, args.dropout)
# encoder2 = Het_NetEn(args.embed_dim, args.dropout)
# # encoder1 = torch.load('../pretrained_hgnn/encoder1.pth')
# # encoder2 = torch.load('../pretrained_hgnn/encoder2.pth')
# classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
# decoder_a = EdgePredictor(args.embed_dim)
# decoder_p = EdgePredictor(args.embed_dim)
# decoder_t = EdgePredictor(args.embed_dim)
# decoder_c = EdgePredictor(args.embed_dim)
# # decoder_a = torch.load('../pretrained_hgnn/decoder_a.pth')
# # decoder_p = torch.load('../pretrained_hgnn/decoder_p.pth')
# # decoder_t = torch.load('../pretrained_hgnn/decoder_t.pth')
# # decoder_c = torch.load('../pretrained_hgnn/decoder_c.pth')

# decoder_list = [decoder_a, decoder_p, decoder_t, decoder_c]

# encoder1.to(device)
# encoder2.to(device)
# classifier.to(device)
# for decoder in decoder_list:
#     decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'recon')

[tensor(0.0003, device='cuda:0', grad_fn=<DivBackward0>)]
[tensor(0.0003, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0028, device='cuda:0', grad_fn=<DivBackward0>)]
[tensor(0.0003, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0028, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0014, device='cuda:0', grad_fn=<DivBackward0>)]
[tensor(0.0003, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0028, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0014, device='cuda:0', grad_fn=<DivBackward0>), tensor(0.0001, device='cuda:0', grad_fn=<DivBackward0>)]
Epoch [1/200], Loss: 1.0877, Loss_De: 0.0109, Loss_cls: 1.3868, Accuracy: 0.2500, Edge Accuracy: [0.9993352066063281, 0.9993026335908172, 0.998967452376725, 0.887268917919645]
class 0: 0.2469387755102041, Class 0: AUC-ROC- 0.5505, F1 Score- 0.4066; class 1: 0.2469387755102041, Class 1: AUC-ROC- 0.5852, F1 Score- 0.4286; class 2: 0.1, Class 2: AUC-ROC- 0.4855, F1 Score- 0.4000; class 3: 0.1, Class 3: AUC-ROC- 0.4401, F1 Sco

([0.23625,
  0.2125,
  0.2,
  0.1925,
  0.2025,
  0.2025,
  0.20875,
  0.21,
  0.225,
  0.23,
  0.22625,
  0.2225,
  0.22875,
  0.2275,
  0.225,
  0.22875,
  0.23,
  0.225,
  0.22375,
  0.225,
  0.23625],
 [0.5065572916666667,
  0.4829125,
  0.47028958333333337,
  0.46349375,
  0.45883125,
  0.46103645833333334,
  0.46817708333333335,
  0.4767385416666666,
  0.48329791666666666,
  0.48777395833333337,
  0.49288333333333334,
  0.49840625,
  0.50229375,
  0.5049020833333334,
  0.5042260416666666,
  0.5048458333333333,
  0.506075,
  0.5064239583333333,
  0.5067458333333333,
  0.507553125,
  0.5092239583333333],
 [0.4015408526051082,
  0.4012461059190031,
  0.401025641025641,
  0.4013306038894575,
  0.40103092783505156,
  0.4015544041450777,
  0.4014492753623188,
  0.4009268795056643,
  0.3997963340122199,
  0.4001016260162602,
  0.4001016260162602,
  0.4008146639511202,
  0.40112359550561794,
  0.4005076142131979,
  0.40081053698074975,
  0.40060667340748235,
  0.4005050505050505,
  0.400

In [8]:
# test_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = '')

In [ ]:
# torch.save(decoder_list[0], '../pretrained_han/decoder_a.pth')
# torch.save(decoder_list[1], '../pretrained_han/decoder_p.pth')
# torch.save(decoder_list[2], '../pretrained_han/decoder_t.pth')
# torch.save(decoder_list[3], '../pretrained_han/decoder_c.pth')
# torch.save(encoder1, '../pretrained_han/encoder1.pth')
# torch.save(encoder2, '../pretrained_han/encoder2.pth')